In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CS229
# ! ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/MyDrive/CS229
NSynth	test  test_classes  train  train_classes  valid  valid_classes


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os

batch_size = 8
learning_rate = 1e-3

transforms = transforms.Compose(
[
    transforms.Resize((480,640)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root='train_classes_large', transform=transforms)
valid_dataset = datasets.ImageFolder(root='valid_classes_large', transform=transforms)
test_dataset = datasets.ImageFolder(root='test_classes_large', transform=transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# def imshow(inp, title=None):
    
#     inp = inp.cpu() if device else inp
#     inp = inp.numpy().transpose((1, 2, 0))
    
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
    
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)
    
images, labels = next(iter(train_dataloader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", out.shape)

# imshow(out, title=[train_dataset.classes[x] for x in labels])

images-size: torch.Size([8, 3, 480, 640])
out-size: torch.Size([3, 484, 5138])


In [4]:
net = models.resnet18(pretrained=True)
net = net.cuda() if device else net
net

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 41.5MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-5)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 128)
#try out this for dropout
net.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.2, training=m.training))
###
net.fc = net.fc.cuda() if device else net.fc

In [ ]:
n_epochs = 10
print_every = 10
valid_loss_min = np.Inf
test_loss_min = np.Inf
val_loss = []
val_acc = []
test_loss = []
test_acc = []
train_loss = []
train_acc = []
total_step = len(train_dataloader)
for epoch in range(n_epochs):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_dataloader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (valid_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(valid_dataloader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()


batch_loss = 0
total_t=0
correct_t=0
net = torch.load_state_dict(torch.load('resnet.pt'))
with torch.no_grad():
    net.eval()
    for data_t, target_t in (test_dataloader):
        data_t, target_t = data_t.to(device), target_t.to(device)
        outputs_t = net(data_t)
        loss_t = criterion(outputs_t, target_t)
        batch_loss += loss_t.item()
        _,pred_t = torch.max(outputs_t, dim=1)
        correct_t += torch.sum(pred_t==target_t).item()
        total_t += target_t.size(0)
    test_acc.append(100 * correct_t/total_t)
    test_loss.append(batch_loss/len(test_dataloader))
    network_learned = batch_loss < test_loss_min
    print(f'test loss: {np.mean(val_loss):.4f}, test acc: {(100 * correct_t/total_t):.4f}\n')

Epoch 0

Epoch [0/5], Step [0/1251], Loss: 5.2472
Epoch [0/5], Step [20/1251], Loss: 5.2452
Epoch [0/5], Step [40/1251], Loss: 5.0957
Epoch [0/5], Step [60/1251], Loss: 4.9905
Epoch [0/5], Step [80/1251], Loss: 5.0235
Epoch [0/5], Step [100/1251], Loss: 4.7527
Epoch [0/5], Step [120/1251], Loss: 4.7164
Epoch [0/5], Step [140/1251], Loss: 4.6235
Epoch [0/5], Step [160/1251], Loss: 4.4956
Epoch [0/5], Step [180/1251], Loss: 4.8876
Epoch [0/5], Step [200/1251], Loss: 4.4396
Epoch [0/5], Step [220/1251], Loss: 4.3532
Epoch [0/5], Step [240/1251], Loss: 4.2975
Epoch [0/5], Step [260/1251], Loss: 4.4277
Epoch [0/5], Step [280/1251], Loss: 4.3786
Epoch [0/5], Step [300/1251], Loss: 4.2477
Epoch [0/5], Step [320/1251], Loss: 4.3776
Epoch [0/5], Step [340/1251], Loss: 3.8886
Epoch [0/5], Step [360/1251], Loss: 3.7669
Epoch [0/5], Step [380/1251], Loss: 4.0178
Epoch [0/5], Step [400/1251], Loss: 3.8131
Epoch [0/5], Step [420/1251], Loss: 3.5303
Epoch [0/5], Step [440/1251], Loss: 3.6322
Epoch [0

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.title("Train-Validation Accuracy")
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('accuracy', fontsize=12)
plt.legend(loc='best')
plt.savefig("train-validation-accuracy")

In [ ]:
def imshow(inp, title=None):
    
    inp = inp.cpu() if device else inp
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
def visualize_model(net, num_images=4):
    images_so_far = 0
    fig = plt.figure(figsize=(15, 10))
    
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        outputs = net(inputs)
        _, preds = torch.max(outputs.data, 1)
        preds = preds.cpu().numpy() if use_cuda else preds.numpy()
        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(2, num_images//2, images_so_far)
            ax.axis('off')
            ax.set_title('predictes: {}'.format(test_dataset.classes[preds[j]]))
            imshow(inputs[j])
            
            if images_so_far == num_images:
                return 

plt.ion()
visualize_model(net)
plt.ioff()